In [94]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [134]:
assessor = pd.read_csv('../data/Assessor_s_Property_Database_FY14-FY18.csv')

# Create Garage Labels

In [135]:
assessor['MBL'] = assessor.MAP.astype(str) +"-"+ assessor.BLOCK.astype(str) +"-"+  assessor.LOT.astype(str) 

In [136]:
garage_df = (
    assessor
    .groupby('MBL')['HAS GARAGE']
    .apply(list)
    .apply(lambda l: 1 if 'Y' in l else 0)
    .reset_index()
    .rename({'HAS GARAGE':'HAS_GARAGE'}, axis =1)
)

garage_df = garage_df[garage_df.HAS_GARAGE == 1]

In [ ]:
garage_df.to_csv('../data/garage.csv')

# Clean Assessor 

In [138]:
assessor = assessor[assessor.FISCAL_YEAR == 2018]

In [139]:
unit_addr = assessor['STREET'].apply(lambda x: x.strip().lower())
owner_addr = assessor['OWNER ADD'].apply(lambda x: ' '.join(str(x).split(' ')[1:]).strip().lower())
assessor['OWNER_IS_TENANT'] = unit_addr == owner_addr

In [140]:
feature_clms = [
    'EXEMPT AMT', 
    'TAX VALUE', 
    'ROOF STRUCTURE DESCRIP',
    'ROOF COVER DESCRIP', 
    'INT WALL 1 DESCRIP',
    'HEAT TYPE',
    'FUEL TYPE',
    'AC TYPE',
    'GRADE DESCRIP',
    'IMPROVE VAL',
    'MBL'
]
feature_clms_clean = [clm.replace(' ', '_') for clm in  feature_clms]

In [141]:
replacement_clms = dict(zip(feature_clms, feature_clms_clean))

In [142]:
assessor = (
    assessor
    .rename(replacement_clms, axis = 1)
    [feature_clms_clean]
)

In [177]:
vc = assessor.INT_WALL_1_DESCRIP.value_counts()
assessor.INT_WALL_1_DESCRIP.apply(lambda x: vc.loc[x] if type(x) == str else np.NaN)

53487        NaN
53512        NaN
53625        NaN
53651        NaN
53739        NaN
          ...   
88905    13850.0
88906       94.0
88907        NaN
88908        NaN
88909        NaN
Name: INT_WALL_1_DESCRIP, Length: 18911, dtype: float64

In [170]:
assessor.sort_values('IMPROVE_VAL', ascending=False).drop_duplicates(subset = ['MBL'])

,EXEMPT_AMT,TAX_VALUE,ROOF_STRUCTURE_DESCRIP,ROOF_COVER_DESCRIP,INT_WALL_1_DESCRIP,HEAT_TYPE,FUEL_TYPE,AC_TYPE,GRADE_DESCRIP,IMPROVE_VAL,MBL
77400,0.0,463108.67,Green Roof,Flat,Drywall/Sheet,Forced Air-Duc,Gas,Central,Excellent,192342000,99-A-9
77256,0.0,1123815.70,T&G/Rubber,Flat,Drywall/Sheet,Forced Air-Duc,Gas,Central,Excellent,76642400,85-A-8
77289,0.0,921109.56,T&G/Rubber,Flat,Drywall/Sheet,Forced Air-Duc,Gas,Central,Average,70190900,85-A-11
59791,0.0,528230.87,T&G/Rubber,Flat,Adequate,Hot Water,Gas,None,Average +20,67024600,115-C-1
75918,0.0,0.00,Flat,T&G/Rubber,Plastered,Forced Air-Duc,Gas,Central,Excellent +10,60997000,61-F-2
...,...,...,...,...,...,...,...,...,...,...,...
80772,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,90-I-22
80704,0.0,109.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,18-A-15
80670,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,40-H-12
80655,0.0,3807.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,83-D-9


In [172]:
assessor.to_csv('../data/assessor_clean.csv')